## Импорт необходимых библиотек

In [1]:
import re
import spacy
import csv
import torch
import nltk

from itertools import chain
from tqdm import tqdm

## Подготовка данных

In [2]:
nlp = spacy.load('ru_core_news_lg')

In [3]:
data_path = "./data/some_data/processed_data/"

In [4]:
with open(data_path + "original_texts.txt", 'r', encoding='utf-8') as f:
    orig_texts = f.read().split('\n')

In [9]:
pipeline = nlp.pipe_names.copy()
pipeline

['tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [10]:
pipeline.remove('parser')
pipeline.remove('tok2vec')
pipeline.remove('lemmatizer')
pipeline.remove('morphologizer')
pipeline

['ner', 'attribute_ruler']

In [11]:
tokenized_lemm_texts = []

In [12]:
for text in tqdm(nlp.pipe(orig_texts, disable=pipeline)):
    lemm_text = []
    for token in text:
        if token.dep_ == 'nsubj':
            lemm_text.append('<PH>')
        else:
            lemm_text.append(token.lemma_.lower())
    tokenized_lemm_texts.append(lemm_text)

360279it [09:45, 615.54it/s]


In [14]:
lemm_texts = []

In [15]:
for text in tqdm(tokenized_lemm_texts):
    assembled = ""
    for token in text:
        assembled += token + ' '
    assembled = re.sub(r'\s([?.!"](?:\s|$))', r'\1', assembled).rstrip()
    lemm_texts.append(assembled)

100%|██████████████████████████████████████████████████████████████████████| 360279/360279 [00:01<00:00, 219633.93it/s]


### Построение csv-файла с необходимыми признаками

In [25]:
pipeline = nlp.pipe_names.copy()
pipeline

['tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [26]:
pipeline.remove('parser')
pipeline.remove('tok2vec')
pipeline.remove('morphologizer')
pipeline

['ner', 'attribute_ruler', 'lemmatizer']

In [28]:
dataset = []

for i, text in tqdm(enumerate((nlp.pipe(orig_texts, disable=pipeline)))):
    for token in text:
        if token.dep_ == 'nsubj':
            dataset.append({
                'orig_texts': orig_texts[i].lower(),
                'lemm_texts': lemm_texts[i].lower(),
                'nsubj': token.text.lower(),
            })
            break

360279it [02:26, 2465.22it/s]


In [38]:
dataset[:5]

[{'orig_texts': 'я предлагаю оригинальный подарок для малыша!',
  'lemm_texts': '<PH> предлагать оригинальный подарок для малыш!',
  'nsubj': 'я'},
 {'orig_texts': 'я обезательно перезвоню в любом случае.',
  'lemm_texts': '<PH> обезательно перезвонить в любой случай.',
  'nsubj': 'я'},
 {'orig_texts': 'цены на память я не помню.',
  'lemm_texts': 'цена на память <PH> не помнить.',
  'nsubj': 'я'},
 {'orig_texts': 'я не помню, где находились.',
  'lemm_texts': '<PH> не помнить, где находиться.',
  'nsubj': 'я'},
 {'orig_texts': 'я работаю на высококачественных американских материалах.',
  'lemm_texts': '<PH> работать на высококачественный американский материал.',
  'nsubj': 'я'}]

In [39]:
with open(data_path + 'dataset.csv', 'w', encoding='utf-8') as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerow(['orig_texts', 'lemm_texts', 'nsubj'])
    for item in tqdm(dataset):
        csv_writer.writerow(item.values())

100%|██████████████████████████████████████████████████████████████████████| 358502/358502 [00:00<00:00, 445856.68it/s]


### Токенизация предложений

In [46]:
tokenized_texts = [nltk.tokenize.wordpunct_tokenize(text.lower()) for text in tqdm(lemm_texts)]

### Определяем класс словаря, в котором каждому токену ставится в соответствие число

In [44]:
class Vocab:
    def __init__(self, tokens, unk_index):
        self._tokens = tokens
        self._token_to_id = { token: i for i, token in enumerate(tokens) }
        self._unk_index = unk_index
        
    def __len__(self):
        return len(self._tokens)
    
    def word_to_id(self, word):
        return self._token_to_id.get(word, self._unk_index)
    
    def id_to_word(self, idx):
        return self._tokens[idx]